In [33]:
# Парсер объявлений на сайте auto.ru по тестовому заданию на стажера тех отдела TrainingData.Solutions

In [34]:
import time
import zipfile
import shutil

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from tqdm import tqdm
import requests

from selenium_stealth import stealth

from selenium.common import TimeoutException
from selenium.common import StaleElementReferenceException, WebDriverException

In [35]:
options = Options()
options.add_argument("--start-maximized")

options.add_argument('--disable-gpu')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

chromedriver = 'chromedriver.exe'
service = ChromeService(executable_path=chromedriver)
webdriver = webdriver.Chrome(options=options, service=service)

stealth(webdriver,
        languages=["en-US", "en"],
        user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

webdriver.get(f'https://auto.ru/')

"""Обходим капчу и всплывающие окна"""
if webdriver.find_elements(By.XPATH, '//*[@id="confirm-button"]'):
    WebDriverWait(webdriver, 5).until(ec.element_to_be_clickable((By.XPATH, '//*[@id="confirm-button"]'))).click()
if webdriver.find_elements(By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'):
    WebDriverWait(webdriver, 5).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'))).click()
time.sleep(3)
if webdriver.find_elements(By.XPATH, '/html/body/div[3]/div/div/div[2]'):
    WebDriverWait(webdriver, 1).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/div/div[2]'))).click()

"""Создаем список из ссылок 10ти первых марок машин на сайте"""

urls = []
car_col = webdriver.find_element(By.CLASS_NAME, f'IndexMarks').find_elements(By.CLASS_NAME, 'IndexMarks__col')
for car_catalog in car_col:
    car_brand_page = car_catalog.find_elements(By.TAG_NAME, 'a')
    for car_link in car_brand_page:
        urls.append(car_link.get_attribute('href'))
urls = urls[:10]

"""Создаем директорию для хранения ссылок и фотографий объявлений"""

for i in range(len(urls)):
    webdriver.get(urls[i])
    car_brand = urls[i].split('/')[4]

    if webdriver.find_elements(By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'):
        WebDriverWait(webdriver, 5).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'))).click()
    if webdriver.find_elements(By.XPATH, '/html/body/div[2]/div/div/div[2]'):
        WebDriverWait(webdriver, 5).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div/div[2]'))).click()
    webdriver.execute_script("window.scrollTo(0, 1080)")

    ad_list = webdriver.find_elements(By.CLASS_NAME, 'ListingItem')
    for j in range(10):
        ad_title = ad_list[j].find_element(By.TAG_NAME, 'h3').text.replace(' ', '_').replace('/', '-')
        screenshot_save_directory = f'result/{car_brand}/{ad_title}_{j}/'

        if not os.path.exists(screenshot_save_directory):
            os.makedirs(screenshot_save_directory)
        time.sleep(0.1)
        WebDriverWait(webdriver, 10).until(ec.visibility_of_any_elements_located((By.CLASS_NAME, 'ListingItem')))[j].screenshot(screenshot_save_directory + f'{ad_title}.png')

    soup = BeautifulSoup(webdriver.page_source, 'html.parser')
    car_brand_links_directory = 'brand_links/'
    if not os.path.exists(car_brand_links_directory):
        os.makedirs(car_brand_links_directory)
    with open(f'{car_brand_links_directory}{car_brand}.html', 'w', encoding="utf-8") as file:
        file.write(str(soup))

webdriver.quit()

In [36]:
"""Отбираем по 10 первых моделей машин каждой марки"""

car_links = []
links_limit = 10
brand_directory = 'brand_links'
if not os.path.exists(brand_directory):
    os.makedirs(brand_directory)
for file in os.listdir(brand_directory):
    soup = BeautifulSoup(open(f'{brand_directory}/{file}', 'r', encoding="utf-8").read(), 'html.parser')
    counter = 0
    for div_class in soup.find_all('div', class_='ListingItem__main'):
        try:
            car_link = div_class.find('a', class_='Link ListingItemTitle__link').get('href')
            car_links.append(car_link)
            counter += 1
            if counter >= links_limit:
                break
        except AttributeError:
            continue

In [37]:
links_limit = 10
titles = []
brand_directory = 'brand_links'
if not os.path.exists(brand_directory):
    os.makedirs(brand_directory)
for file in os.listdir(brand_directory):
    soup = BeautifulSoup(open(f'{brand_directory}/{file}', 'r', encoding="utf-8").read(), 'html.parser')
    counter = 0
    for div_class in soup.find_all('div', class_='ListingItem__main'):
        try:
            title = div_class.find('h3', class_='ListingItemTitle ListingItem__title').text
            titles.append(title)
            counter += 1
            if counter >= links_limit:
                break
        except AttributeError:
            continue

from selenium import webdriver

options = Options()
options.add_argument("--start-maximized")
options.add_argument('--disable-gpu')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

webdriver = webdriver.Chrome(options=options)

stealth(webdriver,
        languages=["en-US", "en"],
        user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

"""В созданную директорию марка машины/модель загружаем до 5ти фото из объявления"""

for i in tqdm(range(len(car_links))):
    try:
        webdriver.get(car_links[i])
        car_brand = car_links[i].split('/')[6]

        if webdriver.find_elements(By.XPATH, '//*[@id="confirm-button"]'):
            WebDriverWait(webdriver, 5).until(ec.element_to_be_clickable((By.XPATH, '//*[@id="confirm-button"]'))).click()
        if webdriver.find_elements(By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'):
            WebDriverWait(webdriver, 5).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'))).click()
        time.sleep(3)
        if webdriver.find_elements(By.XPATH, '/html/body/div[3]/div/div/div[2]'):
            WebDriverWait(webdriver, 1).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/div/div[2]'))).click()

        soup = BeautifulSoup(webdriver.page_source, 'html.parser')

        WebDriverWait(webdriver, 10).until(ec.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/div[2]/div[3]/div/div[2]/div/div[2]/div/div[2]/div[5]/div[2]/div/div[2]/div[1]/div[4]/div/img')))
        if webdriver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div[3]/div/div[2]/div/div[2]/div/div[2]/div[5]/div[2]/div/div[2]/div[1]/div[4]/div/img'):
            WebDriverWait(webdriver, 10).until(ec.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/div[2]/div[3]/div/div[2]/div/div[2]/div/div[2]/div[5]/div[2]/div/div[2]/div[1]/div[4]/div/img'))).click()

        images = webdriver.find_elements(By.CLASS_NAME, 'ImageGalleryFullscreenVertical__image')

        image_count = 0
        image_limit = 5
        reformat_title = str(titles[i]).replace(' ', '_').replace('/', '-')
        if i > 9:
            result_directory = f'result/{car_brand}/{reformat_title}_{(i - 10) % 10}/'  # Добавляем i из-за наличия дубликатов в объявлениях
            if os.path.exists(result_directory):
                for index, image in enumerate(images):
                    image_url = image.get_attribute('src')
                    if image_url:
                        response = requests.get(image_url, stream=True)

                        if not os.path.exists(result_directory):
                            os.makedirs(result_directory)
                        image_path = os.path.join(result_directory + f'{reformat_title}_{index + 1}.jpg')
                        # Сохраняем изображение
                        with open(image_path, 'wb') as file:
                            for chunk in response.iter_content(chunk_size=128):
                                file.write(chunk)
                        image_count += 1
                        if image_count >= image_limit:
                            break

        else:
            result_directory = f'result/{car_brand}/' + f'{reformat_title}_{i}/'  # Добавляем i из-за наличия дубликатов в объявлениях
            if os.path.exists(result_directory):
                for index, image in enumerate(images):
                    image_url = image.get_attribute('src')
                    if image_url:
                        response = requests.get(image_url, stream=True)

                        if not os.path.exists(result_directory):
                            os.makedirs(result_directory)
                        image_path = os.path.join(result_directory + f'{reformat_title}_{index + 1}.jpg')
                        # Сохраняем изображение
                        with open(image_path, 'wb') as file:
                            for chunk in response.iter_content(chunk_size=128):
                                file.write(chunk)
                        image_count += 1
                        if image_count >= image_limit:
                            break

    except TimeoutException:
        continue

webdriver.quit()

100%|██████████| 100/100 [10:28<00:00,  6.29s/it]


In [38]:
"""Добавляем результат в архив"""

result = zipfile.ZipFile('result.zip', 'w')
for root, dirs, files in os.walk('result'):
    for file in files:
        result.write(os.path.join(root, file))

result.close()

In [39]:
"""Удаляем созданные для работы директории"""

shutil.rmtree('result', ignore_errors=False, onerror=None)
shutil.rmtree('brand_links', ignore_errors=False, onerror=None)